### <center> Análise Preditiva: Música Final de ano (RETROSPECTIVA)

## Objetivo: Prever a música que será a mais escutada ao final do ano igual o spotify mostra na retrospectiva. utilizando somente os dados que seria usado para a propia no caso do dia 01/01 para frente, porem vamos pegar um pouco no final do anos ja que temos uma base pequena

In [65]:
# Importações
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from prophet import Prophet


In [66]:
# Lê o csv com o gênero das músicas
df = pd.read_csv('spotify_songs_pedro.csv')

### Preparação de Dados

In [ ]:
# Excluindo colunas desnecessárias
df_music = df.drop(columns=['platform', 'conn_country', 'ip_addr', 'master_metadata_album_album_name', 'spotify_track_uri', 'episode_name', 'episode_show_name', 'spotify_episode_uri', 'audiobook_uri', 'audiobook_title', 'audiobook_chapter_uri', 'audiobook_chapter_title', 'reason_start', 'reason_end', 'offline', 'offline_timestamp', 'incognito_mode', 'genero'])

# Perido que ira ser analisado: 25/11/2025 a diante
df_music = df_music[df_music['ts'] >= '2025-11-25']

display(df_music.head())


,ts,ms_played,master_metadata_track_name,master_metadata_album_artist_name,shuffle,skipped
20928,2025-11-25 17:24:15+00:00,153934,Status Que Eu Não Queria - Ao Vivo,Zé Neto & Cristiano,True,False
20929,2025-11-25 17:27:43+00:00,207520,Pra Sempre Com Você - Ao Vivo,Jorge & Mateus,True,False
20930,2025-11-25 17:30:31+00:00,166491,Manda um Oi - Ao Vivo,Guilherme & Benuto,True,False
20931,2025-11-25 17:32:45+00:00,133547,Barulho Do Foguete - Ao Vivo,Zé Neto & Cristiano,True,False
20932,2025-11-25 17:35:30+00:00,163914,Como É Que A Gente Fica - Ao Vivo,Henrique & Juliano,True,False


In [68]:
# apagar do df as músicas que foram puladas
df_music = df_music[~(df_music['skipped'] == True)]

display(df_music.head())

,ts,ms_played,master_metadata_track_name,master_metadata_album_artist_name,shuffle,skipped
20928,2025-11-25 17:24:15+00:00,153934,Status Que Eu Não Queria - Ao Vivo,Zé Neto & Cristiano,True,False
20929,2025-11-25 17:27:43+00:00,207520,Pra Sempre Com Você - Ao Vivo,Jorge & Mateus,True,False
20930,2025-11-25 17:30:31+00:00,166491,Manda um Oi - Ao Vivo,Guilherme & Benuto,True,False
20931,2025-11-25 17:32:45+00:00,133547,Barulho Do Foguete - Ao Vivo,Zé Neto & Cristiano,True,False
20932,2025-11-25 17:35:30+00:00,163914,Como É Que A Gente Fica - Ao Vivo,Henrique & Juliano,True,False


In [69]:
# Converter timestamp
df_music['ts'] = pd.to_datetime(df_music['ts'])

# Criar colunas úteis
df_music['ano'] = df_music['ts'].dt.year
df_music['mes'] = df_music['ts'].dt.to_period('M')

# Converter ms_played → minutos
df_music['min_played'] = df_music['ms_played'] / 60000

historico_real = df_music.groupby('master_metadata_track_name')['min_played'].sum().reset_index()
historico_real.columns = ['musica', 'minutos_reais']

display(historico_real)

C:\Users\pedroandrade-ieg\AppData\Local\Temp\ipykernel_14568\354948250.py:6: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df_music['mes'] = df_music['ts'].dt.to_period('M')


,musica,minutos_reais
0,#SELFIE,12.250000
1,21 Guns,5.351550
2,21st Century (Digital Boy),2.816883
3,3AM (PXT4 RASA),2.034133
4,5 Da Manhã - Ao Vivo,17.163533
...,...,...
380,rockstar (feat. 21 Savage),7.271567
381,so low,10.459400
382,É Com Ela Que Eu Estou - Ao Vivo,7.361767
383,É Hit - Ao Vivo,30.776550


### Features

In [70]:
# Agregação mensal por música
df_month = (
    df_music.groupby(['mes', 'master_metadata_track_name'])
      ['min_played'].sum()
      .reset_index()
)

df_month['mes'] = df_month['mes'].dt.to_timestamp()

display(df_month.head())

,mes,master_metadata_track_name,min_played
0,2025-11-01,3AM (PXT4 RASA),2.034133
1,2025-11-01,5 Da Manhã - Ao Vivo,2.451933
2,2025-11-01,A Rosa E O Beija-Flor - Na Praia / Ao Vivo,2.847550
3,2025-11-01,Acordando o Prédio,3.641117
4,2025-11-01,Alô Porteiro - Ao Vivo,3.280300


### Modelo

In [ ]:
resultados = []
avaliacoes = []

musicas_validas = df_month['master_metadata_track_name'].unique()

for musica in musicas_validas:
    df_tmp = df_month[df_month['master_metadata_track_name'] == musica][['mes', 'min_played']]
    df_tmp.columns = ['ds', 'y']
    df_tmp = df_tmp.dropna().sort_values('ds')

    if len(df_tmp) < 2:
        continue 

    n_test_eval = 1 
    train = df_tmp.iloc[:-n_test_eval]
    test = df_tmp.iloc[-n_test_eval:]

    # Só calculamos erro se sobrar dado para treino após o split
    if len(train) >= 4:
        model_eval = Prophet(yearly_seasonality=False, weekly_seasonality=False, daily_seasonality=False)
        model_eval.fit(train)
        
        # Prever o período de teste para validar o modelo
        future_eval = model_eval.make_future_dataframe(periods=n_test_eval, freq='ME')
        forecast_eval = model_eval.predict(future_eval)
        
        y_true = test['y'].values
        y_pred = forecast_eval.tail(n_test_eval)['yhat'].values

        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        avaliacoes.append([musica, mae, rmse])

    model_final = Prophet(yearly_seasonality=False, weekly_seasonality=False, daily_seasonality=False)
    model_final.fit(df_tmp)

    # Previsão dos 10 meses futuros
    n_previsao = 10
    future_final = model_final.make_future_dataframe(periods=n_previsao, freq='ME')
    forecast_final = model_final.predict(future_final)

    # Soma dos 10 meses projetados
    previsao_minutos = max(0, forecast_final.tail(n_previsao)['yhat'].sum())
    resultados.append([musica, previsao_minutos])

df_result = pd.DataFrame(resultados, columns=['musica', 'previsao_minutos'])

09:18:39 - cmdstanpy - INFO - Chain [1] start processing
09:18:39 - cmdstanpy - INFO - Chain [1] done processing
09:18:39 - cmdstanpy - INFO - Chain [1] start processing
09:18:41 - cmdstanpy - INFO - Chain [1] done processing
09:18:41 - cmdstanpy - INFO - Chain [1] start processing
09:18:41 - cmdstanpy - INFO - Chain [1] done processing
09:18:42 - cmdstanpy - INFO - Chain [1] start processing
09:18:42 - cmdstanpy - INFO - Chain [1] done processing
09:18:42 - cmdstanpy - INFO - Chain [1] start processing
09:18:42 - cmdstanpy - INFO - Chain [1] done processing
09:18:42 - cmdstanpy - INFO - Chain [1] start processing
09:18:44 - cmdstanpy - INFO - Chain [1] done processing
09:18:44 - cmdstanpy - INFO - Chain [1] start processing
09:18:44 - cmdstanpy - INFO - Chain [1] done processing
09:18:45 - cmdstanpy - INFO - Chain [1] start processing
09:18:45 - cmdstanpy - INFO - Chain [1] done processing
09:18:45 - cmdstanpy - INFO - Chain [1] start processing
09:18:45 - cmdstanpy - INFO - Chain [1]

In [72]:
avaliacoes_ordenadas = sorted(avaliacoes, key=lambda x: x[1], reverse=False)

print(f"{'Música':<50} | {'MAE':^7} | {'RMSE'}")
for avaliacao in avaliacoes_ordenadas:
    print(f"{avaliacao[0]:<50} | {round(avaliacao[1], 4):^7} | {round(avaliacao[2], 4)}")

Música                                             |   MAE   | RMSE


### Resultado do Modelo (Previsão)

In [ ]:
df_result = pd.DataFrame(resultados, columns=['musica', 'previsao_minutos'])

# unir os dados reais com os previstos
df_final = pd.merge(df_result, historico_real, on='musica', how='left')

# preencher com 0 caso alguma música prevista não tenha histórico
df_final['minutos_reais'] = df_final['minutos_reais'].fillna(0)

# soma do que já foi com o que virá
df_final['total_projetado_ano'] = df_final['minutos_reais'] + df_final['previsao_minutos']

# Ordenar para ver o top 1 da Retrospectiva
df_final_ranking = df_final.sort_values('total_projetado_ano', ascending=False)

In [74]:
top_1 = df_final_ranking.head(1)

print("Sua música mais ouvida do ano provavelmente será:")
print(top_1)

Sua música mais ouvida do ano provavelmente será:
        musica  previsao_minutos  minutos_reais  total_projetado_ano
38  Evidências       1567.144061      32.584183          1599.728244
